In [1]:
import cv2
import matplotlib.pyplot as plt
import pyflow.pyflow as pyflow
import numpy as np
from evaluation import MSEN, PEPN

In [2]:
# Code imported from https://github.com/mcv-m6-video/mcv-m6-2023-team2/blob/main/week4/utils_w4.py
def convert_optical_flow_to_image(flow):
    # The 3-channel uint16 PNG images that comprise optical flow maps contain information
    # on the u-component in the first channel, the v-component in the second channel,
    # and whether a valid ground truth optical flow value exists for a given pixel in the third channel.
    # A value of 1 in the third channel indicates the existence of a valid optical flow value
    # while a value of 0 indicates otherwise. To convert the u- and v-flow values from
    # their original uint16 format to floating point values, one can do so by subtracting 2^15 from the value,
    # converting it to float, and then dividing the result by 64.

    img_u = (flow[:, :, 2] - 2 ** 15) / 64
    img_v = (flow[:, :, 1] - 2 ** 15) / 64

    img_available = flow[:, :, 0]  # whether a valid GT optical flow value is available
    img_available[img_available > 1] = 1

    img_u[img_available == 0] = 0
    img_v[img_available == 0] = 0

    optical_flow = np.dstack((img_u, img_v, img_available))
    return optical_flow

In [3]:
first = cv2.imread(r'kitti_data\training\image_0\000045_10.png').astype(float) / 255.
second = cv2.imread(r'kitti_data\training\image_0\000045_11.png').astype(float) / 255.

gt_flow = cv2.imread(r'kitti_data\training\flow_noc\000045_10.png', cv2.IMREAD_UNCHANGED).astype(np.double)
gt_flow_img = convert_optical_flow_to_image(gt_flow)

first.shape, second.shape, gt_flow.shape

((376, 1241, 3), (376, 1241, 3), (376, 1241, 3))

In [20]:
u, v, im2W = pyflow.coarse2fine_flow(first, second)

u.shape, v.shape, im2W.shape

((376, 1241), (376, 1241), (376, 1241, 3), (376, 1241, 2))

In [22]:
msen, sen = MSEN(gt_flow_img, im2W, './results')
pepn = PEPN(sen)

print('MSEN:', msen)
print('PEPN:', pepn)

MSEN: 10.60653739789043
PEPN: 78.69069299338636


<Figure size 1100x400 with 0 Axes>

<Figure size 1100x400 with 0 Axes>